This is based on a tutorial from Susan Li, [published on Medium](https://towardsdatascience.com/sales-forecasting-with-price-promotion-effects-b5d70207b128). The purpose is to "analyze the historical data using time series analysis techniques, with promotion effect."

My goal is to learn how to apply time series to questions around pricing

In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("Sales_Product_Price_by_Store.csv")

Just getting a sense of what is in this dataset

In [8]:
df.head()

,Store,Product,Date,Is_Holiday,Base Price,Price,Weekly_Units_Sold
0,1,1,2/5/2010,False,9.99,7.99,245
1,1,1,2/12/2010,True,9.99,7.99,453
2,1,1,2/19/2010,False,9.99,7.99,409
3,1,1,2/26/2010,False,9.99,7.99,191
4,1,1,3/5/2010,False,9.99,9.99,145


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3861 entries, 0 to 3860
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Store              3861 non-null   int64  
 1   Product            3861 non-null   int64  
 2   Date               3861 non-null   object 
 3   Is_Holiday         3861 non-null   bool   
 4   Base Price         3861 non-null   float64
 5   Price              3861 non-null   float64
 6   Weekly_Units_Sold  3861 non-null   int64  
dtypes: bool(1), float64(2), int64(3), object(1)
memory usage: 184.9+ KB


In [16]:
#do all the stores sell for the same price?
df.groupby(['Store']).mean()

,Product,Is_Holiday,Base Price,Price,Weekly_Units_Sold
Store,,,,,
1,2.0,0.06993,12.554103,11.766084,202.095571
2,2.0,0.06993,12.554103,11.766084,283.135198
3,2.0,0.06993,12.554103,11.766084,73.867133
4,2.0,0.06993,12.554103,11.766084,369.972028
5,2.0,0.06993,12.554103,11.766084,63.636364
6,2.0,0.06993,12.554103,11.766084,227.734266
7,2.0,0.06993,12.554103,11.766084,102.627040
8,2.0,0.06993,12.554103,11.766084,152.151515
10,2.0,0.06993,12.554103,11.766084,468.354312
